In [42]:
import pandas as pd
import re
from datetime import datetime
import os

# Load the data from a CSV file

#file_path = os.path.join(os.path.expanduser("~"), "Downloads")
df = pd.read_csv( 'messy_data.csv')

# Inspect the data
print(df.head())
print(df.info())

# Rename the unnamed column to 'index'
df.rename(columns={df.columns[0]: 'Index'}, inplace=True)
df.to_csv('cleaned_dataset.csv', index=False)
# Handle missing values
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Salary'] = df['Salary'].fillna(df['Salary'].mean())
df.dropna(subset=['Name', 'Email'], inplace=True)

# Remove duplicate rows
df.drop_duplicates(inplace=True)
# Remove duplicate rows based on 'ID', keeping the first occurrence
df = df.drop_duplicates(subset='ID', keep='first')

# Correct email formats
def is_valid_email(email):
    return re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email)

def is_professional_email(email):
    return not any(domain in email for domain in ['hotmail', 'yahoo', 'gmail'])

df['Email'] = df['Email'].apply(lambda x: x if is_valid_email(x) else None)
df.dropna(subset=['Email'], inplace=True)
df = df[df['Email'].apply(is_professional_email)]

# Clean name fields
def clean_name(name):
    return re.sub(r'\b(Mr\.|Mrs\.|Ms\.|Dr\.|DDS|DVM)\b', '', name).strip()

df['Name'] = df['Name'].apply(clean_name)

# Standardize date formats

def parse_date(date_str):
    for fmt in ('%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y'):
        try:
            return datetime.strptime(date_str, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    return '1970-01-01'  # Default value if unable to parse

# Apply the parse_date function to 'Join Date' column
df['Join Date'] = df['Join Date'].apply(lambda x: parse_date(x) if pd.notnull(x) else '1970-01-01')

# Print the DataFrame to check for empty cells in 'Join Date' column
print(df[df['Join Date'].isnull()])

# Save the DataFrame back to a CSV file (optional)
df.to_csv('cleaned_dataset.csv', index=False)

# Correct department names
department_mapping = {
    'hr': 'HR',
    'human resources': 'HR',
    'eng': 'Engineering',
    'engineering': 'Engineering',
    'mktg': 'Marketing',
    'marketing': 'Marketing',
    'sales': 'Sales',
    'support': 'Support',
    # Add other variations as necessary
}

def clean_and_map_department(dept):
    if isinstance(dept, str):
        dept = dept.lower()
        dept = re.sub(r'[^a-z\s]', '', dept)
        corrections = {
            'hr': r'h\s*r|human\s*resources|h\s*r',
            'engineering': r'eng(?:ineering)?|engineeringi+',
            'marketing': r'mktg|marketings?|marketingu+',
            'sales': r'salesx?',
            'support': r'supporte?'
        }

        for correct, pattern in corrections.items():
            if re.search(pattern, dept):
                return correct.capitalize()

        return department_mapping.get(dept, dept.capitalize())
    else:
        return 'Unknown'

df['Department'] = df['Department'].apply(clean_and_map_department)

# Handle salary noise
min_salary = 30000
max_salary = 200000
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
df.loc[(df['Salary'] < min_salary) | (df['Salary'] > max_salary), 'Salary'] = None
df['Salary'] = df['Salary'].fillna(df['Salary'].mean())




# Sort the DataFrame by 'Name' column in ascending order
df_sorted = df.sort_values(by='Name')
df_sorted.to_csv('cleaned_dataset.csv', index=False, float_format='%.0f')




df = pd.read_csv('cleaned_dataset.csv')

# Drop the 'Index' column if it exists
if 'Index' in df.columns:
    df.drop(columns=['Index'], inplace=True)

# Reset the index and drop the old index
df = df.reset_index(drop=True)

# Set the index to start from 1
df.index = df.index + 1

# Save the DataFrame back to a CSV file (optional)
df.to_csv('cleaned_dataset.csv', index=True)






   Unnamed: 0                                    ID                     Name  \
0           0  1e407ff9-6255-489d-a0de-34135d4f74bd            Hunter Thomas   
1           1  379f55b8-87d5-4739-a146-7400b78c24d1             Jeremy Irwin   
2           2  18261368-dfa1-47f0-afc6-bddf45926b07  Jennifer Hammondquickly   
3           3  ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5          Sydney Taylorso   
4           4  14ed3e6a-e0f5-4bbe-8d93-8665267f5c90                Julia Lee   

    Age                    Email   Join Date    Salary   Department  
0  25.0       xlopez@hotmail.com         NaN   88552.0        Sales  
1  90.0          Jillian Jenkins  2022-07-07  139227.0          NaN  
2  66.0          jscottgreen.biz  2023-11-21   65550.0  Engineering  
3  39.0       luke56gonzalez.com  2021-11-05  139932.0     SupportJ  
4  71.0  figueroakayla@yahoo.com         NaN  143456.0    Marketing  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 c